In [ ]:
""" Master File w/ hyperparameter sweeping across multiple architectures"""
"""
Restart kernel after running
Only need to run once
"""
!pip install scikit-learn matplotlib seaborn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import math 
import torch
import torch.utils.data as td
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, Subset, BatchSampler
import wandb

torch.backends.cudnn.benchmark = True

In [2]:
""" Load datasets """
filename = './data/k562_datasets.pkl'

with open(filename, 'rb') as file:
    combined_datasets = pickle.load(file)

train_data = combined_datasets['train']
valid_data = combined_datasets['valid']
test_data = combined_datasets['test']

print(train_data.iloc[0])

column_names = np.array(train_data.columns)
feature_names = column_names[6:16]
num_features = len(feature_names)
print(feature_names)
num_samples = train_data.shape[0]
nucleotides = ['A', 'T', 'G', 'C']
num_seq_features = len(nucleotides)

print("Number of Samples: " + str(num_samples))
print("Number of Features: " + str(num_features))

cuda_available = torch.cuda.is_available()
print("CUDA (GPU support) is available:", cuda_available)
num_gpus = torch.cuda.device_count()
print("Number of GPUs available:", num_gpus)

seqnames                         1
start                      1002760
end                        1002760
strand                           +
ensembl_gene_id    ENSG00000187608
score                          0.0
ctcf                      -0.07771
h4k20me1                 -0.429997
h3k79me2                   -0.2804
h3k4me1                  -0.217665
h3k9me3                  -0.333359
h3k36me3                 -0.801406
sj5                      -0.039619
sj3                      -0.059131
rpts                     -0.187111
wgbs                           0.0
lambda_alphaj             0.026377
zeta                      1.133344
A                                0
T                                0
G                                1
C                                0
dataset                      train
Name: 0, dtype: object
['ctcf' 'h4k20me1' 'h3k79me2' 'h3k4me1' 'h3k9me3' 'h3k36me3' 'sj5' 'sj3'
 'rpts' 'wgbs']
Number of Samples: 136927782
Number of Features: 10
CUDA (GPU support) is available

In [3]:
sweep_config = {
    'method': 'grid'
}
metric = {
    'name': 'valid_neural_net_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'model_type': {
        'values': ['linear']
    },
    'num_lstm_layers': {
        'values': [1]
    },
    'ep_hidden_layer_sizes': {
        'values': [[16]] 
    },
    'ep_filter_size': {
        'values': [9] 
    },
    'ep_dilation_rate': {
        'values': [0] 
    },
    'seq_hidden_layer_sizes': {
        'values': [[16]] 
    },
    'seq_filter_size': {
        'values': [9] 
    },
    'seq_dilation_rate': {
        'values': [0] 
    },
    'dropout': {
        'values': [0.5]
    },
    'lstm_hidden_layer_size': {
        'values': [16]
    },
    'num_lstm_layers': {
        'values': [0]
    },
    'bidirectional': {
        'values': [False]
    },
    'train_use_sliding_window': {
        'values': [True]
    },
    'train_window_size': {
        'values': [100]
    },
    'train_stride': {
        'values': [100]
    },
    'valid_use_sliding_window': {
        'values': [False]
    },
    'valid_window_size': {
        'values': [None]
    },
    'valid_stride': {
        'values': [None]
    },
    'learn_rate': {
        'values': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    },
}

parameters_dict.update({
    'epochs': {
        'value': 20}
    })

sweep_config['parameters'] = parameters_dict

In [4]:
sweep_id = wandb.sweep(sweep_config, project="elongation-net-linear-w100")

NameError: name 'wandb' is not defined

In [ ]:
""" Process data using a sliding window approach """
class GeneDataset(Dataset):
    def __init__(self, dataframe, use_sliding_window=False, window_size=100, stride=25):
        self.dataframe = dataframe
        self.grouped_data = dataframe.groupby('ensembl_gene_id')
        self.use_sliding_window = use_sliding_window
        self.window_size = window_size
        self.stride = stride
        self.cache = {}
        self.windows = []

        # use subsequence windows from genes
        if self.use_sliding_window and window_size is not None:
            self._create_windows()
        # use full-length genes
        else:
            self._prepare_full_genes()
    
    def _create_windows(self):
        for gene_id, group in self.grouped_data:
            gene_length = len(group)
            for start_idx in range(0, gene_length - self.window_size + 1, self.stride):#self.window_size):
                end_idx = start_idx + self.window_size
                if end_idx > gene_length:
                    break
                window = group.iloc[start_idx:end_idx]
                self.windows.append((gene_id, window))
    
    def _prepare_full_genes(self):
        for gene_id, group in self.grouped_data:
            self.windows.append((gene_id, group))

    def __len__(self):
        return len(self.windows)

    # prepare single window or gene
    def __getitem__(self, idx):
        gene_id, window = self.windows[idx]
        
        if gene_id in self.cache:
            return self.cache[gene_id]
        
        strand_encoded = window['strand'].map({'-': 0, '+': 1}).values
        strand_tensor = torch.tensor(strand_encoded, dtype=torch.int64)
 
        result = {
            'GeneId': gene_id,
            'Seq_Name': window['seqnames'].iloc[0],
            'Start': torch.tensor(window['start'].values, dtype=torch.int64),
            'End': torch.tensor(window['end'].values, dtype=torch.int64),
            'Strand': strand_tensor,
            
            # epigenomic features per gene j, site i
            'Y_ji':  torch.tensor(window[feature_names].values, dtype=torch.float64),
            
            # read counts per gene j, site i
            'X_ji': torch.tensor(window['score'].values, dtype=torch.float64),
            
            # read depth * initiation rate values per gene j
            'C_j': torch.tensor(window['lambda_alphaj'].iloc[0], dtype=torch.float64),
            
            # GLM elongation rate predictions per gene j, site i
            'Z_ji': torch.tensor(window['zeta'].values, dtype=torch.float64),
            
            # one-hot encoded sequences
            'N_ji': torch.tensor(window[nucleotides].values, dtype=torch.float64), 
            'Length': len(window)
        }
    
        self.cache[gene_id] = result

        return result

In [ ]:
""" Batch subsequences with same gene id together """
class GeneIdBatchSampler(BatchSampler):
    def __init__(self, dataset):
        self.dataset = dataset
        self.batches = self._create_batches()

    def _create_batches(self):
        # Group indices by GeneId
        gene_id_to_indices = {}
        for idx in range(len(self.dataset)):
            gene_id = self.dataset[idx]['GeneId']
            if gene_id not in gene_id_to_indices:
                gene_id_to_indices[gene_id] = []
            gene_id_to_indices[gene_id].append(idx)

        return list(gene_id_to_indices.values())

    def __iter__(self):
        for batch in self.batches:
            yield batch

    def __len__(self):
        return len(self.batches)

In [ ]:
def build_dataset(train_data, use_sliding_window=False, window_size=100, stride=25):
    dataset = GeneDataset(train_data, use_sliding_window, window_size, stride)
    batch_sampler = GeneIdBatchSampler(dataset)
    loader = DataLoader(dataset, batch_sampler=batch_sampler, shuffle=False, num_workers=7)
    return loader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def build_model(model_type, num_features, num_seq_features, y_hidden_layer_sizes, y_filter_size, y_dilation_rate,
                     n_hidden_layer_sizes, n_filter_size, n_dilation_rate, dropout, weight_init,
                     lstm_hidden_layer_size, num_lstm_layers=None, bidirectional=False):
    
    class LinearModel(nn.Module):
        def __init__(self, input_size):
            super(LinearModel, self).__init__()
            self.name = "linear"
            self.linear = nn.Linear(input_size, 1, bias=False)

        def forward(self, x):
            x = self.linear(x)
            return x.squeeze(-1)
    
    class LSTMModel(nn.Module):
        def __init__(self, input_size, hidden_layer_size, num_layers, bidirectional):
            super(LSTMModel, self).__init__()
            self.name = "lstm"
            self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, bidirectional=bidirectional, batch_first=True)
            self.bidirectional_linear = nn.Linear(2 * hidden_layer_size, 1)
            self.linear = nn.Linear(hidden_layer_size, 1)
            self.bidirectional = bidirectional

        def forward(self, Y_ji, N_ji):
            x = torch.cat((Y_ji, N_ji), axis=-1)
            x, _ = self.lstm(x)
            if self.bidirectional:
                x = self.bidirectional_linear(x)
            else:
                x = self.linear(x)
            return x.squeeze(-1)

    class CNN(nn.Module):
        def __init__(self, num_features, num_seq_features, y_hidden_layer_sizes, y_filter_size, y_dilation_rate,
                     n_hidden_layer_sizes, n_filter_size, n_dilation_rate, dropout, 
                     lstm_hidden_layer_size, num_lstm_layers=None, bidirectional=False):
            super(CNN, self).__init__()
            self.name = "cnn"            

            self.y_convs = nn.ModuleList()
            y_in_channels = num_features
            
            y_dilation = 1
            y_dilation_rate = 2
            for out_channels in y_hidden_layer_sizes:
                y_padding = ((y_filter_size - 1) * y_dilation) // 2
                if y_dilation_rate > 0:
                    self.y_convs.append(
                        nn.Conv1d(y_in_channels, out_channels, y_filter_size, stride=1, padding=y_padding, dilation=y_dilation)
                    )
                    y_dilation_rate *= y_dilation_rate
                else:
                    self.y_convs.append(
                        nn.Conv1d(y_in_channels, out_channels, y_filter_size, stride=1, padding='same')
                    )
                y_in_channels = out_channels
            
            
            self.n_convs = nn.ModuleList()
            n_in_channels = num_seq_features

            n_dilation = 1
            n_dilation_rate = 2
            for out_channels in n_hidden_layer_sizes:
                n_padding = ((n_filter_size - 1) * n_dilation) // 2
                if n_dilation_rate > 0:
                    self.n_convs.append(
                        nn.Conv1d(n_in_channels, out_channels, n_filter_size, stride=1, padding=n_padding, dilation=n_dilation)
                    )
                    n_dilation_rate *= n_dilation_rate
                else:
                    self.n_convs.append(
                        nn.Conv1d(n_in_channels, out_channels, n_filter_size, stride=1, padding='same')
                    )
                n_in_channels = out_channels

            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(dropout)

            # Final convolutional layer to map to a single output channel
            # Since the output needs to be (batch_size, seq_len), we map the final features to 1
            self.final_conv = nn.Conv1d(y_hidden_layer_sizes[-1] + n_hidden_layer_sizes[-1], 1, 1)  # 1x1 convolution
            
            if num_lstm_layers > 0:
                self.gru = nn.GRU(input_size=y_hidden_layer_sizes[-1] + n_hidden_layer_sizes[-1], hidden_size=lstm_hidden_layer_size, num_layers=num_lstm_layers, bidirectional=bidirectional, batch_first=True)
            self.final_linear = nn.Linear(lstm_hidden_layer_size, 1)
            self.bidirectional = True
            self.final_bidirectional_linear = nn.Linear(lstm_hidden_layer_size*2, 1)
            
        def forward(self, Y_ji, N_ji):
            Y_ji = Y_ji.permute(0, 2, 1)  
            N_ji = N_ji.permute(0, 2, 1)
            
            for conv in self.y_convs:
                Y_ji = conv(Y_ji)
                Y_ji = self.relu(Y_ji)
                Y_ji = self.dropout(Y_ji)
            
            for conv in self.n_convs:
                N_ji = conv(N_ji)
                N_ji = self.relu(N_ji)
                N_ji = self.dropout(N_ji)

            x = torch.cat((Y_ji, N_ji), 1)
            
            if num_lstm_layers > 0:
                x = x.permute(0,2,1)
                x, (h_n, c_n) = self.gru(x)
                if self.bidirectional:
                    x = self.final_bidirectional_linear(x)
                else:
                    x = self.final_linear(x)
                x = x.squeeze(-1)
            
            else:
                x = self.final_conv(x)
                x = x.squeeze(1)  
                
            return x
    
    if model_type == 'lstm':
        model = LSTMModel(num_features + num_seq_features, lstm_hidden_layer_sizes, num_lstm_layers, bidirectional)
    elif model_type == 'linear':
        model = LinearModel(num_features)
    elif model_type == 'cnn':
        model = CNN(num_features, num_seq_features, y_hidden_layer_sizes, y_filter_size, y_dilation_rate, 
                    n_hidden_layer_sizes, n_filter_size, n_dilation_rate, dropout, lstm_hidden_layer_size,
                    num_lstm_layers, bidirectional)
    
    if cuda_available:
        if num_gpus > 1:
            print("Using", num_gpus, "GPUs")
            Dmodel = torch.nn.DataParallel(model)
        model = model.to('cuda')

    print(model)
    
    # expected weights are close to 0 which is why 0 initializing weights converges much quicker
    if weight_init == 'zero':
        with torch.no_grad():
            for param in model.parameters():
                param.zero_()
    
    model.double()

    return model.to(device)

In [ ]:
def build_optimizer(network, learning_rate):
    optimizer = optim.Adam(network.parameters(), lr=learning_rate)
    return optimizer

In [ ]:
def valid_epoch(model, loader, loss_fn):
    model.eval()
    total_neural_net_loss = 0
    total_glm_loss = 0
    neural_net_zeta = []
    glm_zeta = []
    
    with torch.no_grad():
        for idx, batch in enumerate(loader):
            Y_ji_batch = batch['Y_ji'].to(device)
            X_ji_batch = batch['X_ji'].to(device)
            N_ji_batch = batch['N_ji'].to(device) 
            C_j_batch = batch['C_j'].to(device).unsqueeze(1)
            Z_ji_batch = batch['Z_ji'].to(device)
            
            outputs = model(Y_ji_batch, N_ji_batch)
            
            neural_net_loss = loss_fn(X_ji_batch, C_j_batch, outputs)
            glm_loss = loss_fn(X_ji_batch, C_j_batch, torch.log(Z_ji_batch))

            total_neural_net_loss +=  neural_net_loss.item()
            total_glm_loss += glm_loss.item()
            
            # store all predictions in list
            neural_net_zeta.append(torch.exp(outputs.cpu()[0]))
            glm_zeta.append(batch['Z_ji'][0])
    
    # calculate average loss across all batches
    avg_neural_net_loss = total_neural_net_loss / len(loader)
    avg_glm_loss = total_glm_loss / len(loader)
    
    neural_net_zeta = torch.cat(neural_net_zeta, dim=0)
    glm_zeta = torch.cat(glm_zeta, dim=0)
    
    return avg_neural_net_loss, avg_glm_loss, neural_net_zeta, glm_zeta

In [ ]:
def train_epoch(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for idx, batch in enumerate(loader):
        optimizer.zero_grad(set_to_none=True)
        Y_ji_batch = batch['Y_ji'].to(device) 
        X_ji_batch = batch['X_ji'].to(device)
        N_ji_batch = batch['N_ji'].to(device) 
        C_j_batch = batch['C_j'].to(device).unsqueeze(1)
        
        outputs = model(Y_ji_batch, N_ji_batch)
        
        loss = loss_fn(X_ji_batch, C_j_batch, outputs)
        loss.backward()
        optimizer.step()
        
        # calculate average loss across all batches
        total_loss += loss.item()
    avg_train_loss = total_loss / len(loader)
    
    return avg_train_loss

In [ ]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, X_ji, C_j, rho_ji):
        loss = X_ji * rho_ji + C_j * torch.exp(-rho_ji) - X_ji * torch.log(C_j)
        
        # calculate average loss within each batch
        return (loss).mean()

In [ ]:
increase_cut=0.0001

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        model = build_model(config.model_type, num_features, num_seq_features, config.ep_hidden_layer_sizes, 
                            config.ep_filter_size, config.ep_dilation_rate, config.seq_hidden_layer_sizes, 
                            config.seq_filter_size, config.seq_dilation_rate, config.dropout, None, 
                            config.lstm_hidden_layer_size, config.num_lstm_layers, config.bidirectional)
        
        train_loader = build_dataset(train_data, config.train_use_sliding_window, config.train_window_size, config.train_stride)
        valid_loader = build_dataset(valid_data, config.valid_use_sliding_window, config.valid_window_size, config.valid_stride)
        
        optimizer = build_optimizer(model, config.learn_rate)
        
        loss_fn = torch.jit.script(CustomLoss())
        loss_neural_net_train = [0] * config.epochs
        loss_neural_net_valid = [0] * config.epochs
        loss_glm_valid = [0] * config.epochs
        
        # scheduler to reduce learning rate by half when new validation loss > old validation loss
        old_neural_net_valid_loss = float('inf')
        learning_rate_decreased = False
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, verbose=True)

        for epoch in range(config.epochs):
            print(f'Epoch {epoch+1}')
            
            train_loss = train_epoch(model, train_loader, optimizer, loss_fn)
            loss_neural_net_train[epoch] = train_loss
            print(f"train loss: {train_loss: .4f}")
            
            valid_neural_net_loss, valid_glm_loss, neural_net_zeta, glm_zeta = valid_epoch(model, valid_loader, loss_fn)
            loss_neural_net_valid[epoch] = valid_neural_net_loss
            loss_glm_valid[epoch] = valid_glm_loss
            print(f"valid neural net loss: {valid_neural_net_loss: .6f}")
            print(f"valid glm loss: {valid_glm_loss: .6f}")
            
            # calculate metrics
            mae = F.l1_loss(neural_net_zeta.squeeze(), glm_zeta)
            mse = F.mse_loss(neural_net_zeta.squeeze(), glm_zeta)
            correlation_coefficient = np.corrcoef(glm_zeta, neural_net_zeta.squeeze())[0, 1]
            print("Correlation Coefficient:", correlation_coefficient)
            print(f"Mean Absolute Error: {mae.item():.4f}")
            print(f"Mean Squared Error: {mse.item():.4f}")
            
            
            wandb.log({"epoch": epoch, "train_loss": train_loss, "valid_neural_net_loss": valid_neural_net_loss,
                       "valid_glm_loss": valid_glm_loss})#, "corr_coeff": correlation_coefficient, "mae": mae.item(), 
                       "mse": mse.item()})
            
            # early stopping if loss is not improving after reducing learning rate
            if learning_rate_decreased and valid_neural_net_loss - old_neural_net_valid_loss < increase_cut:
                break
                
            # reduce learning rate if new loss > old loss
            learning_rate_decreased = False
            if valid_neural_net_loss > old_neural_net_valid_loss:
                optimizer.param_groups[0]['lr'] *= 0.5
                print(f"Reduced learning rate to {optimizer.param_groups[0]['lr']}")
                learning_rate_decreased=True
            old_train_loss = train_loss
            scheduler.step(train_loss)
            
        return model

In [ ]:
wandb.agent(sweep_id, train)